In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path

artifact_torch_root = Path().absolute().parent.parent

os.chdir(artifact_torch_root)

Path().absolute()

In [ ]:
import pandas as pd
import seaborn as sns
from artifact_core.table_comparison import TabularDataSpec
from artifact_experiment import DataSplit
from artifact_experiment.tracking import FilesystemTrackingClient
from artifact_torch.base.data.data_loader import DataLoader
from artifact_torch.table_comparison.routine import TableComparisonRoutineData
from matplotlib import pyplot as plt

from demos.table_comparison.config.constants import (
    BATCH_SIZE,
    DROP_LAST,
    EXPERIMENT_ID,
    LS_CAT_FEATURES,
    LS_CTS_FEATURES,
    N_BINS_CTS,
    SHUFFLE,
    TRAINING_DATASET_PATH,
)
from demos.table_comparison.data.dataset import DemoDataset
from demos.table_comparison.experiment.experiment import DemoTabularSynthesisExperiment
from demos.table_comparison.libs.transformers.discretizer import Discretizer
from demos.table_comparison.libs.transformers.encoder import Encoder
from demos.table_comparison.model.protocols import TabularVAEInput, TabularVAEOutput
from demos.table_comparison.model.synthesizer import TabularVAESynthesizer

In [ ]:
sns.set_theme(style="whitegrid", palette="colorblind")

In [ ]:
df_real = pd.read_csv(artifact_torch_root / TRAINING_DATASET_PATH)

raw_data_spec = TabularDataSpec.from_df(
    df=df_real,
    ls_cts_features=LS_CTS_FEATURES,
    ls_cat_features=LS_CAT_FEATURES,
)

df_real

In [ ]:
discretizer = Discretizer(n_bins=N_BINS_CTS, ls_cts_features=raw_data_spec.ls_cts_features)

discretizer.fit(df=df_real)

df_discretized = discretizer.transform(df=df_real)

df_discretized

In [ ]:
encoder = Encoder()

encoder.fit(df=df_discretized, ls_cat_features=list(df_discretized.columns))

df_encoded = encoder.transform(df=df_discretized)

encoded_data_spec = TabularDataSpec.from_df(df=df_encoded, ls_cat_features=list(df_encoded.columns))

df_encoded

In [ ]:
data_loaders = {
    DataSplit.TRAIN: DataLoader(
        dataset=DemoDataset(df_raw=df_real, discretizer=discretizer, encoder=encoder),
        batch_size=BATCH_SIZE,
        drop_last=DROP_LAST,
        shuffle=SHUFFLE,
    )
}

In [ ]:
artifact_routine_data = {DataSplit.TRAIN: TableComparisonRoutineData(df_real=df_real)}

In [ ]:
model = TabularVAESynthesizer.build(
    data_spec=encoded_data_spec, discretizer=discretizer, encoder=encoder
)

In [ ]:
from artifact_torch.base.components.callbacks.data_loader import DataLoaderCallbackResources
from artifact_torch.base.model.io import LossOutput, ModelInput
from artifact_torch.libs.components.callbacks.data_loader.loss import LoaderLossCallback

loader: DataLoader[ModelInput] = 5

LoaderLossCallback(5, DataSplit.TRAIN).execute(
    resources=DataLoaderCallbackResources[ModelInput, LossOutput](
        step=5, model=model, data_loader=loader
    )
)

In [ ]:
LoaderLossCallback(5, DataSplit.TRAIN).execute(
    resources=DataLoaderCallbackResources[TabularVAEInput, TabularVAEOutput](
        step=5, model=model, data_loader=data_loaders[DataSplit.TRAIN]
    )
)

In [ ]:
tracking_client = FilesystemTrackingClient.build(experiment_id=EXPERIMENT_ID)

In [ ]:
experiment = DemoTabularSynthesisExperiment[TabularVAEInput, TabularVAEOutput].build(
    model=model,
    data_loaders=data_loaders,
    artifact_routine_data=artifact_routine_data,
    artifact_routine_data_spec=raw_data_spec,
    tracking_client=tracking_client,
)

In [ ]:
experiment.run()

In [ ]:
plt.figure(figsize=(10, 6))
experiment.epoch_scores["loss_train"].plot(color=sns.color_palette("colorblind")[2], linewidth=2)
plt.title("Train Loss", fontsize=16, fontweight="bold")
plt.xlabel("Epoch", fontsize=14)
plt.ylabel("Training Loss", fontsize=14)
plt.grid(True, linestyle="--", alpha=0.6)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()